In [3]:
import pandas as pd

# 读取 annotations.csv
annotations = pd.read_csv('/Users/aloez/Downloads/annotations.csv')
print(annotations.head())  # 查看前几行数据
print(annotations.columns)  # 查看列名
print(annotations.shape)  # 查看数据的行数和列数

# 读取 documents.csv
documents = pd.read_csv('/Users/aloez/Downloads/documents.csv')
print(documents.head())
print(documents.columns)
print(documents.shape)

# 读取 rlc_test.csv
rlc_test = pd.read_csv('/Users/aloez/Downloads/rlc_test.csv')
print(rlc_test.head())
print(rlc_test.columns)
print(rlc_test.shape)

# 读取 sentences.csv
sentences = pd.read_csv('/Users/aloez/Downloads/sentences.csv')
print(sentences.head())
print(sentences.columns)
print(sentences.shape)

       id  sentence_id              tag               quote  \
0    7944         1002            ortho           окружющей   
1  101216         1006          agrcase        промышленным   
2   18818         1009           syntax         воздействую   
3  119778         1009     ortho,altern  желудожно-кишечный   
4   18819         1012  syntax,transfer                 это   

           correction  start  end annotation_source  
0          окружающей     12   13            manual  
1        промышленных      4    5            manual  
2         воздействуя     12   13            manual  
3  желудочно-кишечный     17   18            manual  
4                          7    8            manual  
Index(['id', 'sentence_id', 'tag', 'quote', 'correction', 'start', 'end',
       'annotation_source'],
      dtype='object')
(41410, 8)
   id subcorpus native language_background level  words  sentences
0   1     RULEC    eng                  HL    AM    431         22
1   3     RULEC    eng     

In [7]:
# 假设 tag 含 'genitive' 代表属格错误，筛选这类数据，添加 na=False 处理空值
genitive_errors = annotations[annotations['tag'].str.contains('genitive', case=False, na=False)]
# 同理筛选宾格错误，添加 na=False
accusative_errors = annotations[annotations['tag'].str.contains('accusative', case=False, na=False)]

# 统计数量
genitive_count = genitive_errors.shape[0]
accusative_count = accusative_errors.shape[0]
print(f"属格错误数量：{genitive_count}，宾格错误数量：{accusative_count}")

属格错误数量：0，宾格错误数量：0


In [11]:
# 设置 pandas 显示选项，强制完整输出
pd.set_option('display.max_rows', None)
print(annotations['tag'].unique())
print(annotations['tag'].unique().tolist())

['ortho' 'agrcase' 'syntax' ... 'hyphen+ins' 'nominative' 'hyphen+del']
['ortho', 'agrcase', 'syntax', 'ortho,altern', 'syntax,transfer', 'infl', 'infl,subst', 'gov', 'constr,transfer', 'altern', 'lex,transfer', 'ref', 'constr', 'lex', 'wo', 'conj', nan, 'brev', 'morph', 'ortho,space', 'transfer,infl', 'num', 'transfer', 'deriv,refl', 'punct', 'asp,morph', 'agrpers', 'num,transfer', 'par', 'space', 'ortho,subst', 'tense,punct', 'ref,subst,transfer', 'ref,deriv', 'lex,not-clear', 'constr,asp', 'lex,prep,gov,transfer', 'agrnum', 'asp', 'com', 'hyphen', 'lex,ortho', 'conj,miss,space,extra', 'conj,miss', 'ortho,miss', 'ortho,not-clear', 'agrgender', 'lex,constr,gov', 'syntax,transfer,conj,wo', 'lex,constr,transfer,idiom,prep', 'deriv', 'constr,conj', 'wo,transfer', 'ref,transfer', 'translit', 'transfer,lex', 'gov,constr', 'misspell', 'subst,constr', 'not-clear', 'transfer,syntax', 'wo,syntax', 'constr,syntax', 'conj,space', 'constr,lex', 'idiom,transfer', 'agrcase,gov', 'ortho,misspell', '

In [12]:
nominative_errors = annotations[annotations['tag'] == 'nominative']
print("主格错误数量：", nominative_errors.shape[0])

主格错误数量： 91


In [ ]:
# 1. 改写判断逻辑：用俄语语法判断属格/宾格错误（示例逻辑，需你补充实际语法规则）
def is_genitive_or_accusative(row):
    # 示例：根据俄语词尾判断（如属格词尾 -а/-я，宾格词尾 -у/-ю 等，需结合实际调整）
    wrong = row['quote']
    correct = row['correction']
    # 替换为你的俄语语法判断逻辑，返回 True/False
    return any(ending in wrong for ending in ['-а', '-я']) and any(ending in correct for ending in ['-у', '-ю'])  

# 2. 用 .loc 避免警告，重新处理数据
agrcase_errors = annotations[annotations['tag'] == 'agrcase'].copy()  # 显式拷贝，断开与原数据的关联
agrcase_errors.loc[:, 'is_gen_acc'] = agrcase_errors.apply(is_genitive_or_accusative, axis=1)

# 3. 统计数量
gen_acc_count = agrcase_errors[agrcase_errors['is_gen_acc']].shape[0]
print(f"格一致错误中属格/宾格相关错误数量：{gen_acc_count}")

格一致错误中属格/宾格相关错误数量：0


In [14]:
# 假设从错误内容判断属格、宾格错误，这里简单模拟（需结合实际语法判断）
def is_genitive_or_accusative(row):
    # 结合 quote 和 correction，用俄语语法判断是否是属格/宾格错误
    # 示例：若包含属格/宾格词尾差异逻辑，返回 True
    return 'genitive' in row['quote'].lower() or 'accusative' in row['quote'].lower()  # 实际需替换为语法判断

agrcase_errors['is_gen_acc'] = agrcase_errors.apply(is_genitive_or_accusative, axis=1)
gen_acc_count = agrcase_errors[agrcase_errors['is_gen_acc']].shape[0]
print(f"格一致错误中属格/宾格相关错误数量：{gen_acc_count}")

格一致错误中属格/宾格相关错误数量：0


/var/folders/8h/b0lq6_nn1_n6wpvbmw5r5b500000gn/T/ipykernel_73508/955335886.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agrcase_errors['is_gen_acc'] = agrcase_errors.apply(is_genitive_or_accusative, axis=1)


In [17]:
print(agrcase_errors[['quote', 'correction']].head(20)) 

                  quote                correction
1          промышленным              промышленных
180                 том                       тот
362         обоих стран             обеим странам
450            входящие                  входящую
451               любой                     любую
537                  он                        им
538                того                       тем
540              работе                    работу
556                сами                     самим
643   обычные рестораны      в обычных ресторанах
651          популярные               популярными
679                СПИД                     СПИДа
686                СПИД                     СПИДа
708            В статьи                  В статье
722               жизни                   о жизни
723       администрации            администрацией
726         всю историю               вся история
749             распаха                    распах
776     межкультурности        "межкультурность")


In [19]:
def is_genitive_or_accusative(row):
    wrong = row['quote']
    correct = row['correction']
    genitive_endings = ['-ым', '-ом', '-их', '-ей']
    accusative_endings = ['-ой', '-ий', '-ом', '-ем']
    return (any(wrong.endswith(end) for end in genitive_endings) and 
            any(correct.endswith(end) for end in genitive_endings) or 
            any(wrong.endswith(end) for end in accusative_endings) and 
            any(correct.endswith(end) for end in accusative_endings))

agrcase_errors['is_gen_acc'] = agrcase_errors.apply(is_genitive_or_accusative, axis=1)
gen_acc_count = agrcase_errors[agrcase_errors['is_gen_acc']].shape[0]
print(f"格一致错误中属格/宾格相关错误数量：{gen_acc_count}")

格一致错误中属格/宾格相关错误数量：0


In [21]:
import pandas as pd
import numpy as np

# 1. 随机排序数据
agrcase_errors = agrcase_errors.sample(frac=1, random_state=42)  # random_state固定随机种子，确保可复现


# 3. 定义词类分层比例（假设统计结果）
word_class_ratio = {'noun': 0.6, 'adj': 0.3, 'pron': 0.1}

# 4. 按比例抽取各层样本
stratified_samples = []
for word_class, ratio in word_class_ratio.items():
    class_data = merged_data[merged_data['word_class'] == word_class]
    sample_size = int(len(agrcase_errors) * ratio)
    stratified_samples.append(class_data.head(sample_size))

# 5. 合并分层样本
final_sample = pd.concat(stratified_samples)

# 6. 补充抽取低频错误（如属格错误）
rare_errors = merged_data[merged_data['case_type'] == 'genitive']
if len(rare_errors) < 10:  # 假设属格错误占比＜10%
    final_sample = pd.concat([final_sample, rare_errors])

# 7. 去重并导出
final_sample = final_sample.drop_duplicates('sentence_id').head(100)  # 控制在100条
final_sample[['sentence_id', 'quote', 'correction']].to_excel('格错误标注样本.xlsx', index=False)

NameError: name 'merged_data' is not defined

In [ ]:
import pandas as pd

# --- 1. 加载数据集 ---
print("--- 1. 加载数据集 ---")
try:
    documents_df = pd.read_csv('/Users/aloez/Downloads/documents.csv')
    annotations_df = pd.read_csv('/Users/aloez/Downloads/annotations.csv')
    print("数据集加载成功！")
    print(f"documents.csv 包含 {len(documents_df)} 条记录。")
    print(f"annotations.csv 包含 {len(annotations_df)} 条记录。")

except FileNotFoundError:
    print("错误：请确保 'documents.csv' 和 'annotations.csv' 文件在当前目录下或提供正确路径。")
    exit() # 如果文件未找到，程序退出

# --- 2. 初步查看数据结构 ---
print("\n--- 2. 初步查看 documents.csv 的前几行 ---")
print(documents_df.head())
print("\n--- 2. 初步查看 annotations.csv 的前几行 ---")
print(annotations_df.head())

# 检查列名，确认是否存在 'native' 和 'tag'
print("\n--- documents.csv 的列名 ---")
print(documents_df.columns.tolist())
print("\n--- annotations.csv 的列名 ---")
print(annotations_df.columns.tolist())

# 请根据实际输出，确认 `document_id` 或 `text_id` 等用于关联两个表的公共ID字段
# 通常会有一个唯一的ID来连接文档信息和其对应的错误标注
# 假设是 'document_id'，如果不是，请根据实际列名修改
document_id_col_doc = 'document_id' # 假设 documents.csv 中的文档ID列名
document_id_col_anno = 'document_id' # 假设 annotations.csv 中的文档ID列名

if document_id_col_doc not in documents_df.columns:
    print(f"\n警告：在 documents.csv 中未找到列 '{document_id_col_doc}'。请检查实际列名。")
if document_id_col_anno not in annotations_df.columns:
    print(f"警告：在 annotations.csv 中未找到列 '{document_id_col_anno}'。请检查实际列名。")


# --- 3. 筛选目标学习者数据 ---
print("\n--- 3. 筛选目标学习者数据 ---")

# 筛选母语为汉语 ('zho') 和英语 ('eng') 的学习者
# 论文中通常使用ISO 639-2/3 代码，'zho'代表Chinese，'eng'代表English。
# 请注意：如果您的数据集中使用了不同的编码（例如 'Chinese', 'English'），请修改这里的列表。
target_languages = ['zho', 'eng']
target_learners_df = documents_df[documents_df['native'].isin(target_languages)]

print(f"筛选出 {len(target_learners_df)} 条来自目标母语学习者的记录。")
print("目标学习者的母语分布：")
print(target_learners_df['native'].value_counts())

# 提取目标学习者的 document_id (或其他唯一标识符)
target_document_ids = target_learners_df[document_id_col_doc].unique()
print(f"提取出 {len(target_document_ids)} 个目标学习者的唯一文档ID。")


# --- 4. 在 annotations 文件中定位动词体（Asp）错误 ---
print("\n--- 4. 在 annotations 文件中定位动词体（Asp）错误 ---")

# 查找所有 tag 字段为 'Asp' 的错误记录
# 根据论文，'Asp' 是动词体的标签。
# 如果初步检查中未出现，请务必在运行此代码后，手动检查 annotations.csv 的完整内容，
# 或尝试搜索其他可能的动词体相关标签（如'Aspect', 'VerbAspect'等）。
aspect_errors_df = annotations_df[annotations_df['tag'] == 'Asp'].copy() # 使用.copy()避免SettingWithCopyWarning

print(f"在 annotations.csv 中找到 {len(aspect_errors_df)} 条 'Asp' 错误记录。")
if len(aspect_errors_df) == 0:
    print("警告：未找到任何 'Asp' 标签的错误。请再次确认 'annotations.csv' 中的 'tag' 列是否存在此标签，或是否存在其他表示动词体的标签。")
    print("annotations.csv 中 tag 列出现的前10个标签类型：")
    print(annotations_df['tag'].value_counts().head(10)) # 打印前10个最常见的tag，帮助您识别

# --- 5. 关联目标学习者数据与动词体错误标注 ---
print("\n--- 5. 关联目标学习者数据与动词体错误标注 ---")

# 将 aspect_errors_df 进一步筛选，只保留属于目标学习者的错误
target_aspect_errors_df = aspect_errors_df[
    aspect_errors_df[document_id_col_anno].isin(target_document_ids)
].copy() # 使用.copy()避免SettingWithCopyWarning

print(f"最终筛选出 {len(target_aspect_errors_df)} 条来自目标学习者的动词体（Asp）错误。")

if len(target_aspect_errors_df) > 0:
    print("\n来自目标学习者的动词体错误示例（前5条）：")
    print(target_aspect_errors_df.head())

    # （可选）合并 learning_native 信息到错误记录中，方便后续分析
    # 确保合并时使用的 document_id 是共通的
    merged_errors_df = pd.merge(
        target_aspect_errors_df,
        target_learners_df[[document_id_col_doc, 'native']],
        left_on=document_id_col_anno,
        right_on=document_id_col_doc,
        how='left'
    )
    # 删除重复的 document_id 列，如果它们名称不同的话
    if document_id_col_doc != document_id_col_anno:
        merged_errors_df = merged_errors_df.drop(columns=[document_id_col_doc])

    print("\n合并学习者母语信息后的动词体错误示例（前5条）：")
    print(merged_errors_df.head())

    # 查看这些错误的母语分布
    print("\n目标学习者动词体错误的母语分布：")
    print(merged_errors_df['native'].value_counts())

    # 现在 merged_errors_df 包含了您需要的所有信息，可以保存以便后续分析
    merged_errors_df.to_csv('target_aspect_errors.csv', index=False, encoding='utf-8-sig')
    print("\n已将筛选出的目标动词体错误保存到 'target_aspect_errors.csv'。")

else:
    print("\n没有找到来自目标学习者的动词体（Asp）错误。请检查标签名称是否正确或数据集内容。")